[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seagarwu/mynotebooks/blob/main/shioaji/shioaji_onlinedoc/shioaji-advanced_guide-non_blocking_mode.ipynb)

# Non-blocking Mode

## What is Non-blocking?
First lets see what is Blocking? A function is blocking if it has to wait for something to complete. Yes, every function is blocking — no matter if you are doing I/O or doing CPU task. Everything takes some time. If a function is doing some task which is making the CPU work, then it is blocking the function from returning. Similarly, if a function is trying to get something from the database, then it is going to wait for the result to come and will block until then to continue the processing.

Conversely, when you make a non-blocking operation, it will never wait for the operation to complete. Non-blocking mode is extreamly useful if you try to send a bulk orders in a short period of time. To make you understand what's difference between blocking and non-block mode, we show you examples of comparison.

## Shioaji Blocking & Non-blocking Mode
Shioaji provides blocking and non-blocking mode, by simply changing the value of timeout. Default value of timeout is 5000 (ms) which means the function will wait for the response up to 5 seconds. If the execution time exceeds 5 seconds, the function will raise error. However, some api function like tick may takes more than 5 seconds. At this circumstance, just scale up timeout or use non-blocking mode by setting the `timeout = 0`.

### Non-Blocking Place Order
To submit order in non-blocking mode is easy, just set `timeout = 0` in `place_order` function.

In [1]:
import shioaji as sj
import getpass as gp

api = sj.Shioaji()
user_id, user_pass = gp.getpass("USER ID: "), gp.getpass("USER PASS: ")
accounts = api.login(user_id, user_pass)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [2]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

api.activate_ca(
    ca_path = file_path,
    ca_passwd = user_pass,
    person_id = user_id,
)

True

In [3]:
# futures default account
api.set_default_account(accounts[2])
print(api.futopt_account)

person_id='R120732106' broker_id='F002002' account_id='8071378' signed=True username='吳鼎榮'


In [ ]:
contract = api.Contracts.Futures.MXF['MXF202205']
order = api.Order(
    action='Sell',
    price=17000,
    quantity=1,
    price_type='LMT',
    order_type='ROD', 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account
)
trade = api.place_order(contract, order, timeout=0)
print(trade)

contract=Future(code='MXFE2', symbol='MXF202205', name='小型臺指05', category='MXF', delivery_month='202205', delivery_date='2022/05/18', underlying_kind='I', unit=1, limit_up=18239.0, limit_down=14923.0, reference=16581.0, update_date='2022/05/03') order=Order(action=<Action.Sell: 'Sell'>, price=17000, quantity=1, account=FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8071378', signed=True, username='吳鼎榮'), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>) status=OrderStatus(status=<Status.Inactive: 'Inactive'>)
OrderState.FOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': 'd1ae41b4', 'seqno': '424185', 'ordno': 'tA0NL', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': '8071378', 'signed': True}, 'action': 'Sell', 'price': 17000.0, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'market_type': 'Night', 'oc_type': 'New', 'subaccount': '', 'combo': Fal

Note that the Trade object is different from the one we get from blocking mode. First, there is no `id` and `seqno` in Order. Second, `id`, `status code`, `order_datetime` and `deals` are missing in OrderStatus. Finally, status is `Inactive`. That's because we hadn't really send the order to exchange. The order is still in transmission. Of course we know nothing about this order. Then it comes to a problem, how do we know this order is successfully submitted? We offer you two kinds of methods, `order event callback` and `non-blocking place order callback`.

以non-blocking下單，可以發現status的部份，會是inactive，表示這筆委託單還沒送到交易所，就是還在傳送中的意思。

#### [Order event callback](https://sinotrade.github.io/tutor/advanced/nonblock/#order-event-callback)
當status的部份有了id跟exchange_ts，表示已經送到交易所了。

#### Non-blocking place order callback

In [4]:
from shioaji.order import Trade

def non_blocking_cb(trade:Trade):
    print('__my_callback__')
    print(trade)

contract = api.Contracts.Futures.MXF['MXF202205']
order = api.Order(
    action='Sell',
    price=17000,
    quantity=1,
    price_type='LMT',
    order_type='ROD', 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account
)

trade = api.place_order(
    contract, 
    order, 
    timeout=0, 
    cb=non_blocking_cb # only work in non-blocking mode
)

__my_callback__
contract=Future(code='MXFE2', symbol='MXF202205', name='小型臺指05', category='MXF', delivery_month='202205', delivery_date='2022/05/18', underlying_kind='I', unit=1, limit_up=18119.0, limit_down=14825.0, reference=16472.0, update_date='2022/05/04') order=Order(action=<Action.Sell: 'Sell'>, price=17000, quantity=1, id='7bead8f6', seqno='467842', account=Account(account_type=<AccountType.Future: 'F'>, person_id='R120732106', broker_id='F002002', account_id='8071378', signed=True), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>) status=OrderStatus(id='7bead8f6', status=<Status.PendingSubmit: 'PendingSubmit'>, status_code='    ', order_datetime=datetime.datetime(2022, 5, 4, 23, 11, 49), deals=[])
OrderState.FOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '7bead8f6', 'seqno': '467842', 'ordno': 'tA09w', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': '8071378', 'sig

### Blocking & Non-Blocking Comparison

In [5]:
# contract and order
contract = api.Contracts.Futures.MXF['MXF202205']
order = api.Order(
    action='Sell',
    price=17000,
    quantity=1,
    price_type='LMT',
    order_type='ROD', 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account
)

In [ ]:
# Blocking
import time
start_time = time.time()
api.place_order(contract, order) # block and wait for the order response
print(time.time() - start_time)
# 0.136578369140625 <- may be different 時間較長

0.0429987907409668
OrderState.FOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': 'dd8c75d4', 'seqno': '468127', 'ordno': 'tA0AK', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': '8071378', 'signed': True}, 'action': 'Sell', 'price': 17000.0, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'market_type': 'Night', 'oc_type': 'New', 'subaccount': '', 'combo': False}, 'status': {'id': 'dd8c75d4', 'exchange_ts': 1651677584, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 1, 'web_id': 'Z'}, 'contract': {'security_type': 'FUT', 'code': 'MXF', 'exchange': 'TIM', 'delivery_month': '202205', 'delivery_date': '', 'strike_price': 0.0, 'option_right': 'Future'}}
OrderState.FOrder {'operation': {'op_type': 'Cancel', 'op_code': '00', 'op_msg': ''}, 'order': {'id': 'dd8c75d4', 'seqno': '468127', 'ordno': 'tA0AK', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': '8

In [8]:
# Non-Blocking
import time
start_time = time.time()
api.place_order(contract, order, timeout=0) # non-block, the order is in transmition (inactive).
print(time.time() - start_time)
# 0.011670351028442383 <- may be different 時間較短

0.005987644195556641
OrderState.FOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '8d6728b1', 'seqno': '468169', 'ordno': 'tA0AP', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': '8071378', 'signed': True}, 'action': 'Sell', 'price': 17000.0, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'market_type': 'Night', 'oc_type': 'New', 'subaccount': '', 'combo': False}, 'status': {'id': '8d6728b1', 'exchange_ts': 1651677642, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 1, 'web_id': 'Z'}, 'contract': {'security_type': 'FUT', 'code': 'MXF', 'exchange': 'TIM', 'delivery_month': '202205', 'delivery_date': '', 'strike_price': 0.0, 'option_right': 'Future'}}
OrderState.FOrder {'operation': {'op_type': 'Cancel', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '8d6728b1', 'seqno': '468169', 'ordno': 'tA0AP', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': 

As you cas see above, in non-blocking mode, it takes about 0.01 seconds to execute `place_order` function, which is 12x faster than the execution time in blocking mode. Remind again, though it's more efficient to place order in non-blocking mode, the order won't be active until the exchange confirms the order.

non-blocking mode發送完成的速度快了許多，不過這是送出委託單的時間，交易所確認的時間會是另外一個時間。

### Non-Blocking mode Supported Function
- Function:
  - ticks
  - place_order
  - update_order
  - cancel_order
  - update_status
  - list_positions
- Parameter: timeout
- Parameter type: int
- Default Mode: Block

---
### Prev: [Order & Deal Event / Futures](shioaji-order_deal_event-futures.ipynb) Next: [Advanced Guide / Touch Price Order](shioaji-advanced_guide-touch_price_order.ipynb)